<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [43]:
import tmdbsimple as tmdb
tmdb.API_KEY = '18b1935c920c9ee0b39b7378364784f6'

In [44]:
movie = tmdb.Movies(778)

In [45]:
print(movie.info())

{'adult': False, 'backdrop_path': '/zF95knRgouUrBNbUh5oqRI5NIcE.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 35, 'name': 'Comedy'}], 'homepage': '', 'id': 778, 'imdb_id': 'tt0046487', 'original_language': 'fr', 'original_title': 'Les vacances de Monsieur Hulot', 'overview': 'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.', 'popularity': 4.943548, 'poster_path': '/r4F4tsU0Ajeh9ZYUkWOJSYmioj7.jpg', 'production_companies': [{'name': 'Cady', 'id': 478}, {'name': 'Discina', 'id': 479}], 'production_countries': [{'iso_3166_1': 'FR', 'name': 'France'}], 'release_date': '1953-02-26', 'revenue': 0, 'runtime': 87, 'spoken_languages': [{'i

In [46]:
movie.info()['overview']

'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.'

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [53]:
import pandas as pd

In [54]:
tmdb_movies = pd.read_csv("movies_info.tsv", delimiter='\t', names=['id', 'title', 'overview', 'genres', 'reviews'], index_col=0)
tmdb_movies.head(2)

,title,overview,genres,reviews
id,,,,
2,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",[]
3,Shadows in Paradise,"An episode in the life of Nikander, a garbage ...","[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",[]


In [55]:
tmdb_movies.index = pd.to_numeric(tmdb_movies.index, errors='coerce')
tmdb_movies.dropna(inplace=True)

In [56]:
import ast
import numpy as np
import scipy
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import scipy.spatial.distance as ssd

In [57]:
movie_text = {}

def extract_genres(s):
    text = ""
    try:
        genres = ast.literal_eval(s)
        for g in genres:
            text = text + " " + g['name']
    except KeyboardInterrupt:
        raise
    except:
        pass
    return text

def extract_reviews(s):
    text = ""
    try:
        reviews = ast.literal_eval(s)
        for r in reviews:
            text = text + " " + r['content']
    except KeyboardInterrupt:
        raise
    except: 
        pass
    return text

for idx, row in tmdb_movies.iterrows():    
    
    text = " ".join([row.title, row.overview, extract_genres(row.genres), 
                     extract_reviews(row.reviews)])
    
    movie_text[int(idx)] = text

In [58]:

movie_text_items = movie_text.items()
movie_text_values = [x[1] for x in movie_text_items]

matrix_to_tmdb_id = {} # matrix ~ X
for i, val in enumerate(movie_text_items):
    matrix_to_tmdb_id[i] = val[0]

In [59]:
vectorizer = TfidfVectorizer(max_df=0.8, max_features=500,
                             min_df=0.01, stop_words='english',
                             use_idf=True)
Xs = vectorizer.fit_transform(movie_text_values).toarray()

X = {}

for i in range(len(Xs)):
    X[matrix_to_tmdb_id[i]] = Xs[i]

In [61]:
X[100]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.22250924,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.20863495,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [63]:
count_vectorizer = CountVectorizer(stop_words='english', ngram_range=(2, 2))
bigrams_s = count_vectorizer.fit_transform(movie_text_values)

bigrams = {}

for i in range(len(Xs)):
    bigrams[matrix_to_tmdb_id[i]] = bigrams_s[i]

In [64]:

print(bigrams[2].shape)

(1, 547198)


In [65]:
count_vectorizer.vocabulary_

{'ariel taisto': 26920,
 'taisto kasurinen': 476406,
 'kasurinen finnish': 263645,
 'finnish coal': 182289,
 'coal miner': 89097,
 'miner father': 318149,
 'father just': 171855,
 'just committed': 261994,
 'committed suicide': 93987,
 'suicide framed': 470233,
 'framed crime': 189689,
 'crime did': 107987,
 'did commit': 128898,
 'commit jail': 93848,
 'jail starts': 254049,
 'starts dream': 458229,
 'dream leaving': 141162,
 'leaving country': 278583,
 'country starting': 103777,
 'starting new': 458116,
 'new life': 337102,
 'life escapes': 282728,
 'escapes prison': 157418,
 'prison things': 380096,
 'things don': 485756,
 'don planned': 138165,
 'planned drama': 368418,
 'drama crime': 140329,
 'shadows paradise': 434935,
 'paradise episode': 354487,
 'episode life': 156006,
 'life nikander': 283246,
 'nikander garbage': 339474,
 'garbage man': 197927,
 'man involving': 302690,
 'involving death': 251136,
 'death worker': 119502,
 'worker affair': 538028,
 'affair drama': 13370,
 

In [66]:
bigrams[100].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [67]:
bigrams[100].shape

(1, 547198)

In [68]:
bigrams[100].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [71]:
X[114]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.30565706,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [72]:
bigrams[114].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [73]:
def key_by_val(d, value):
    for k, v in d.items():
        if v == value:
            return k

# встречающиеся в фильме биграммы
print([key_by_val(count_vectorizer.vocabulary_, i) for i, bigram_n in enumerate(bigrams[100].toarray()[0]) if bigram_n > 0])

['barrels card', 'beautifully cast', 'brilliantly shot', 'card shark', 'cash quick', 'cast cleverly', 'cleverly produced', 'comedy crime', 'crime just', 'decide pull', 'door comedy', 'enlisted friends', 'far good', 'favorite movies', 'filmography genuinely', 'flat door', 'flaws pointing', 'friends need', 'gang happen', 'genuinely favorite', 'good movie', 'happen operating', 'heist small', 'just plain', 'lock stock', 'losing sketchy', 'lot cash', 'love movie', 'make lot', 'match decide', 'movie actually', 'movie far', 'movie tons', 'movies time', 'need make', 'operating flat', 'pattern ritchie', 'plain love', 'pointing pattern', 'poker match', 'produced movie', 'pull heist', 'quick losing', 'realised written', 'recently realised', 'ritchie filmography', 'shark unwillingly', 'shot beautifully', 'sketchy poker', 'small time', 'smoking barrels', 'stock smoking', 'time gang', 'time watched', 'tons flaws', 'unwillingly enlisted', 'watched recently', 'written brilliantly']


## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [ ]:
# put your code here